In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
word_df=pd.read_csv('5-letter.csv', index_col=0)
print(word_df.head())
word_list = word_df.word.iloc[0:5000,].to_list()
#print(word_list)
print(len(word_list))


     word       count
25  WHICH  3140226612
33  THEIR  2152980325
40  THERE  1623700147
46  WOULD  1472811049
51  OTHER  1383185827
5000


In [3]:
np.where(word_df.word=="SUBSP")

(array([5403], dtype=int64),)

In [105]:
class Wordle:
    def __init__(self, word=None, n_guesses=6, n_letters=5, all_words=['PROOF','SHEAR','BUMPY']):
        if word == None:
            self.word = random.sample(set(all_words), 1)[0]
        else:
            self.word = word.upper()
        self.n_guesses = n_guesses
        self.n_letters = n_letters
        self.guess_counter = n_guesses
        self.greens = [None] * self.n_letters 
        self.current_yellows = [None] * self.n_letters
        self.yellows = {i: [] for i in range(self.n_letters)}
        self.guessed_words = []
        self.all_words = all_words
        self.absent_letters = []
        self.present_letters = []
        self.correct_positions = [None] * self.n_letters # greens would be differnt per guess, remember correct positions
        self.state = None
        self.qstate = [0] * self.n_letters
        
    def try_word (self, guess):
        # need to add a check for not doing anything if the word is already guessed correctly
        guess = guess.upper()
        self.state = None
        self.current_yellows = [None]* self.n_letters # yellows have to reset each time
        self.greens = [None] * self.n_letters
        print('guessed: ', guess)
        if not len(guess) == self.n_letters:
            raise ValueError('wrong word length')
        if guess not in self.all_words:
            raise ValueError('invalid word')
        if guess in self.guessed_words:
            raise ValueError('word already guessed')
        self.guessed_words.append(guess)
        # c_g = character_guess,  c_w = character_word
        for i, (c_g, c_w) in enumerate(zip(guess, self.word)):
            if c_g == c_w:
                self.greens[i] = c_g
                self.correct_positions[i] = c_g
                self.qstate[i] = 1
            # Check for number of non-None in greens list 
            if sum(bool(char) for char in self.greens) == self.n_letters:
                self.state = True
                return 'victory! word is: ' + self.word
                
            if c_g in self.word and c_g != c_w:
                self.yellows[i].append(c_g)
                self.current_yellows[i] = c_g
            if c_g in self.word: # track the letters that are there for the AI later
                self.present_letters.append(c_g)
            if c_g not in self.word: # track letters that are not there for the AI
                self.absent_letters.append(c_g)
        self.guess_counter -= 1
        grn_sum = sum([x!=None for x in self.greens])
        yel_sum = sum([x!=None for x in self.current_yellows])
        
        if self.guess_counter == 0:
            self.state=True
            print("YOU LOSE-TOO MANY GUESSES")
            return grn_sum, yel_sum # return number of greens and number of yellows
        print('Greens:' + str(self.greens) + 
              '\nYellows: ' + str(self.yellows) + 
              '\nGuesses left: ' + str(self.guess_counter))

        return grn_sum, yel_sum # return number of greens and number of yellows
        

In [106]:
w = Wordle(all_words = word_list)

In [107]:
# player agent class

## this is where we can have different 'types' of word guesses to be optimized
class Player:
    def __init__(self, all_words):
        self.guesses=0
        self.guessed_words = []
        self.all_words = all_words
        
    def get_random_guess(self):
        '''
        make a guess randomly from the word list
        '''
        
        
        current_guess = random.sample(set(self.all_words), 1)[0]
        self.guessed_words.append(current_guess)
        return current_guess
    def get_common_guess(self):
        i=0
        while( self.all_words[i] in self.guessed_words):
            i+=1
        return self.all_words[i]
    #def guess_by_green(self):
        
        
    #def guess

In [108]:
from gym import spaces

In [109]:
## need an environment class that lets the player interact with wordl
class Environment:
    REWARD_YELLOW = 1
    REWARD_GREEN = 10
    REWARD_WIN = 100
    REWARD_LOSE = -500
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.game_over = False
        self.score = 0
        self.player = Player(all_words = word_list)
        self.wordle = Wordle(all_words=word_list)
        print(self.wordle.word)
        self.guesses = 0
        self.action_space = [0,1]
        
    def step(self, action):
        self.guesses += 1
        
        if action==0:
            current_guess = self.player.get_random_guess()
        elif action == 1:
            current_guess = self.player.get_common_guess()
        
        n_greens, n_yellows = self.wordle.try_word(current_guess)
        reward = 0
        
        if n_greens>0:
            reward += self.REWARD_GREEN * n_greens
            self.score +=self.REWARD_GREEN
        if n_yellows>0:
            reward += self.REWARD_YELLOW * n_yellows
            self.score +=self.REWARD_YELLOW * n_yellows
            
        if self.wordle.state == 'WIN':
            reward += self.REWARD_WIN
            self.score +=self.REWARD_WIN
            return state, reward, self.wordle.state
        
        result = self.wordle.qstate
        
        return result, reward, self.wordle.state
        
        

In [110]:
e = Environment()

HOMES


In [111]:
e.step(0)

guessed:  NORSE
Greens:[None, 'O', None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: ['S'], 4: ['E']}
Guesses left: 5


([0, 1, 0, 0, 0], 12, None)

In [64]:
import gym
import itertools
import matplotlib
import matplotlib.style
import numpy as np
import pandas as pd
import sys
  
  
from collections import defaultdict

import plotting

In [65]:
import gym_gridworlds

In [66]:
env = gym.make('Gridworld-v0')

In [67]:
type(env.action_space)

gym.spaces.discrete.Discrete

In [31]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [68]:
def createEpsilonGreedyPolicy(Q, epsilon, num_actions):
	"""
	Creates an epsilon-greedy policy based
	on a given Q-function and epsilon.
	
	Returns a function that takes the state
	as an input and returns the probabilities
	for each action in the form of a numpy array
	of length of the action space(set of possible actions).
	"""
	def policyFunction(state):

		Action_probabilities = np.ones(num_actions, dtype = float) * epsilon / num_actions
		best_action = np.argmax(Q[state])
		Action_probabilities[best_action] += (1.0 - epsilon)
		return Action_probabilities

	return policyFunction


In [69]:
env.action_space.n

4

In [70]:
Q = defaultdict(lambda: np.zeros(env.action_space.n))

In [71]:
Q[]

SyntaxError: invalid syntax (<ipython-input-71-9921850256af>, line 1)

In [72]:
e = Environment()

GABON


In [73]:
e.action_space

[0, 1]

In [78]:
import plottingepi

In [114]:
def qLearning(env, num_episodes, discount_factor = 1.0, alpha = 0.6, epsilon = 0.1):
    """
    Q-Learning algorithm: Off-policy TD control.
    Finds the optimal greedy policy while improving
    following an epsilon-greedy policy"""
    
    # Action value function
    # A nested dictionary that maps
    # state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(len(env.action_space)))

    # Keeps track of useful statistics
    stats = plottingepi.EpisodeStats(
        episode_lengths = np.zeros(num_episodes),
        episode_rewards = np.zeros(num_episodes))	
    
    # Create an epsilon greedy policy function
    # appropriately for environment action space
    policy = createEpsilonGreedyPolicy(Q, epsilon, len(env.action_space) )
    
    # For every episode
    for ith_episode in range(num_episodes):

        # Reset the environment and pick the first action
        state = env.reset()
        
        for t in itertools.count():
            print("this is q", Q)
            # get probabilities of all actions from current state
            action_probabilities = policy(state)
            print(action_probabilities)
            # choose action according to
            # the probability distribution
            action = np.random.choice(np.arange(
                    len(action_probabilities)),
                    p = action_probabilities)
            print('this is the action', action)
            # take action and get reward, transit to next state
            next_state, reward, done = env.step(action)
            print(sum(next_state))
            next_state = sum(next_state)
            # Update statistics
            stats.episode_rewards[ith_episode] += reward
            stats.episode_lengths[ith_episode] = t

            # TD Update
            best_next_action = np.argmax(Q[next_state])	
            td_target = reward + discount_factor * Q[next_state][best_next_action]
            td_delta = td_target - Q[state][action]
            Q[state][action] += alpha * td_delta

            # done is True if episode terminated
            if done:
                break

            state = next_state

    return Q, stats


In [116]:
q1, s1 = qLearning(Environment(), 10)

SPRAY
VOTED
this is q defaultdict(<function qLearning.<locals>.<lambda> at 0x000001C1E4CADB88>, {})
[0.95 0.05]
this is the action 0
guessed:  NOBLE
Greens:[None, 'O', None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: ['E']}
Guesses left: 5
1
this is q defaultdict(<function qLearning.<locals>.<lambda> at 0x000001C1E4CADB88>, {None: array([6.6, 0. ]), 1: array([0., 0.])})
[0.95 0.05]
this is the action 1
guessed:  WHICH
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: ['E']}
Guesses left: 4
1
this is q defaultdict(<function qLearning.<locals>.<lambda> at 0x000001C1E4CADB88>, {None: array([6.6, 0. ]), 1: array([0., 0.])})
[0.95 0.05]
this is the action 0
guessed:  GLARE
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: ['E', 'E']}
Guesses left: 3
1
this is q defaultdict(<function qLearning.<locals>.<lambda> at 0x000001C1E4CADB88>, {None: array([6.6, 0. ]), 1: array([0.6, 0. ])})
[0.95 0.05]
this is the action 0
guessed: 

ValueError: word already guessed